### Template para análise de dados em tempo real com Apache Pinot

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
from pinotdb import connect
import time

In [2]:
%matplotlib inline 
plt.style.use('ggplot')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
# Configuração de conexão com o Pinot
PINOT_HOST = 'localhost'
PINOT_PORT = 8099
PINOT_TABLE = 'vendas_REALTIME'

In [4]:
# Estabelece a conexão
conn = connect(host=PINOT_HOST, port=PINOT_PORT, path='/query/sql', scheme='http')
cursor = conn.cursor()

In [5]:
# Função auxiliar para executar consultas SQL e retornar um DataFrame
def query_pinot(sql):
    cursor.execute(sql)
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(cursor.fetchall(), columns=columns)
    return df

In [ ]:
# Verificar número total de registros
query = f"SELECT COUNT(*) AS total_registros FROM {PINOT_TABLE}"
total = query_pinot(query)
print(f"Total de registros: {total['total_registros'].values[0]}")

In [ ]:
# Amostra dos primeiros registros
query = f"SELECT * FROM {PINOT_TABLE} LIMIT 5"
amostra = query_pinot(query)
amostra

In [ ]:
# Análise por categoria
query = f"""
SELECT 
    categoria,
    COUNT(*) AS num_vendas,
    SUM(valor_total) AS valor_total,
    AVG(valor_total) AS valor_medio,
    SUM(quantidade) AS quantidade_total
FROM {PINOT_TABLE}
GROUP BY categoria
ORDER BY valor_total DESC
"""

categorias = query_pinot(query)
categorias

In [ ]:
# Visualização das vendas por categoria
plt.figure(figsize=(12, 6))
sns.barplot(x='categoria', y='valor_total', data=categorias)
plt.title('Valor Total de Vendas por Categoria')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Vendas por categoria (gráfico de pizza)
plt.figure(figsize=(10, 10))
plt.pie(categorias['valor_total'], labels=categorias['categoria'], autopct='%1.1f%%', startangle=90)
plt.axis('equal')  # Para garantir que o gráfico de pizza seja um círculo
plt.title('Distribuição de Vendas por Categoria')
plt.show()

In [ ]:
# Análise temporal (agrupada por minuto)
query = f"""
SELECT 
    DATETRUNC('MINUTE', fromEpochMillis(timestamp)) AS minuto,
    COUNT(*) AS num_vendas,
    SUM(valor_total) AS valor_total
FROM {PINOT_TABLE}
GROUP BY DATETRUNC('MINUTE', fromEpochMillis(timestamp))
ORDER BY minuto ASC
"""

vendas_tempo = query_pinot(query)
vendas_tempo['minuto'] = pd.to_datetime(vendas_tempo['minuto'])
print(vendas_tempo)

In [ ]:
# Visualização temporal
plt.figure(figsize=(14, 6))
plt.plot(vendas_tempo['minuto'], vendas_tempo['valor_total'], marker='o', linestyle='-')
plt.title('Valor de Vendas ao Longo do Tempo (por Minuto)')
plt.xlabel('Minuto')
plt.ylabel('Valor Total (R$)')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Análise por estado
query = f"""
SELECT 
    estado,
    COUNT(*) AS num_vendas,
    SUM(valor_total) AS valor_total,
    AVG(valor_total) AS valor_medio
FROM {PINOT_TABLE}
GROUP BY estado
ORDER BY valor_total DESC
"""

estados = query_pinot(query)
estados

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='estado', y='valor_total', data=estados)
plt.title('Valor Total de Vendas por Estado')
plt.tight_layout()
plt.show()

In [ ]:
# Análise por cidade (top 10)
query = f"""
SELECT 
    cidade,
    COUNT(*) AS num_vendas,
    SUM(valor_total) AS valor_total,
    AVG(valor_total) AS valor_medio
FROM {PINOT_TABLE}
GROUP BY cidade
ORDER BY valor_total DESC
LIMIT 10
"""

cidades = query_pinot(query)
cidades

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='cidade', y='valor_total', data=cidades)
plt.title('Valor Total de Vendas por Cidade (Top 10)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Análise por forma de pagamento
query = f"""
SELECT 
    forma_pagamento,
    COUNT(*) AS num_vendas,
    SUM(valor_total) AS valor_total,
    AVG(valor_total) AS valor_medio
FROM {PINOT_TABLE}
GROUP BY forma_pagamento
ORDER BY valor_total DESC
"""

pagamentos = query_pinot(query)
pagamentos

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='forma_pagamento', y='valor_total', data=pagamentos)
plt.title('Valor Total de Vendas por Forma de Pagamento')
plt.tight_layout()
plt.show()

In [ ]:
# Função para análise em tempo real
def analise_tempo_real(intervalo=5, num_iteracoes=20):
    plt.figure(figsize=(14, 10))
    
    for i in range(num_iteracoes):
        # Limpar o gráfico para a próxima iteração
        plt.clf()
        
        # Consultar dados atualizados - Últimos 10 minutos
        query = f"""
        SELECT 
            DATETRUNC('MINUTE', fromEpochMillis(timestamp)) AS minuto,
            COUNT(*) AS num_vendas,
            SUM(valor_total) AS valor_total
        FROM {PINOT_TABLE}
        WHERE timestamp > {int((datetime.now() - timedelta(minutes=10)).timestamp() * 1000)}
        GROUP BY DATETRUNC('MINUTE', fromEpochMillis(timestamp))
        ORDER BY minuto ASC
        """
        
        vendas_recentes = query_pinot(query)
        
        if len(vendas_recentes) == 0:
            plt.text(0.5, 0.5, "Sem dados nos últimos 10 minutos", 
                     horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)
            plt.pause(intervalo)
            continue
        
        vendas_recentes['minuto'] = pd.to_datetime(vendas_recentes['minuto'])
        
        # Atualizar gráfico de vendas por minuto
        plt.subplot(2, 1, 1)
        plt.plot(vendas_recentes['minuto'], vendas_recentes['valor_total'], marker='o', linestyle='-', color='blue')
        plt.title(f'Valor de Vendas - Últimos 10 minutos (Atualização: {datetime.now().strftime("%H:%M:%S")})')
        plt.xlabel('Minuto')
        plt.ylabel('Valor Total (R$)')
        plt.grid(True)
        plt.xticks(rotation=45)
        
        # Consultar dados por categoria (últimos 10 minutos)
        query = f"""
        SELECT 
            categoria,
            COUNT(*) AS num_vendas,
            SUM(valor_total) AS valor_total
        FROM {PINOT_TABLE}
        WHERE timestamp > {int((datetime.now() - timedelta(minutes=10)).timestamp() * 1000)}
        GROUP BY categoria
        ORDER BY valor_total DESC
        """
        
        categorias_recentes = query_pinot(query)
        
        # Atualizar gráfico de categorias
        plt.subplot(2, 1, 2)
        plt.bar(categorias_recentes['categoria'], categorias_recentes['valor_total'], color='green')
        plt.title('Valor Total por Categoria - Últimos 10 minutos')
        plt.xlabel('Categoria')
        plt.ylabel('Valor Total (R$)')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.draw()
        plt.pause(intervalo)
    
    plt.show()

# Executar análise em tempo real (20 iterações com intervalo de 5 segundos)
analise_tempo_real(intervalo=5, num_iteracoes=20)

In [ ]:
# Top 5 produtos mais vendidos
query = f"""
SELECT 
    produto,
    categoria,
    COUNT(*) AS num_vendas,
    SUM(quantidade) AS quantidade_total,
    SUM(valor_total) AS valor_total
FROM {PINOT_TABLE}
GROUP BY produto, categoria
ORDER BY quantidade_total DESC
LIMIT 5
"""

top_produtos = query_pinot(query)
top_produtos

In [ ]:
# Valor médio de venda por hora do dia
query = f"""
SELECT 
    HOUR(fromEpochMillis(timestamp)) AS hora,
    COUNT(*) AS num_vendas,
    AVG(valor_total) AS valor_medio,
    SUM(valor_total) AS valor_total
FROM {PINOT_TABLE}
GROUP BY HOUR(fromEpochMillis(timestamp))
ORDER BY hora ASC
"""

vendas_hora = query_pinot(query)
vendas_hora

In [ ]:
# Visualização por hora do dia
plt.figure(figsize=(12, 6))
plt.plot(vendas_hora['hora'], vendas_hora['valor_total'], marker='o', linestyle='-')
plt.title('Valor Total de Vendas por Hora do Dia')
plt.xlabel('Hora')
plt.ylabel('Valor Total (R$)')
plt.grid(True)
plt.xticks(range(24))
plt.tight_layout()
plt.show()

In [ ]:
# Relação entre categoria e forma de pagamento
query = f"""
SELECT 
    categoria,
    forma_pagamento,
    COUNT(*) AS num_vendas,
    SUM(valor_total) AS valor_total
FROM {PINOT_TABLE}
GROUP BY categoria, forma_pagamento
ORDER BY categoria, valor_total DESC
"""

categ_pagamento = query_pinot(query)
categ_pagamento_pivot = categ_pagamento.pivot_table(index='categoria', columns='forma_pagamento', values='valor_total', aggfunc='sum')
categ_pagamento_pivot.fillna(0, inplace=True)
categ_pagamento_pivot

In [ ]:
# Visualização da relação entre categoria e forma de pagamento
plt.figure(figsize=(14, 8))
sns.heatmap(categ_pagamento_pivot, annot=True, fmt='.0f', cmap='YlGnBu')
plt.title('Valor Total por Categoria e Forma de Pagamento')
plt.tight_layout()
plt.show()